In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D
from keras.layers import  Dropout, Activation
from tensorflow.keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from tensorflow.keras.utils import plot_model
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.utils import CustomObjectScope

from osgeo import gdal
import rasterio as rio
from pyproj import Proj, transform
from tqdm import tqdm
from shapely.geometry import Polygon

import random
import cv2
from random import shuffle
from glob import glob

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [ ]:
from keras.models import load_model
dependencies = {
    'iou': iou
}
model = load_model("Riau.keras", custom_objects=dependencies)

In [ ]:
def read_image2(path):
    #path = path.decode()
    x = gdal.Open(path).ReadAsArray()
    x = np.rollaxis(x, 0, 3)
    x = x.astype(np.float32)
    return x

def read_mask2(path):
    #path = path.decode()
    x = gdal.Open(path).ReadAsArray()
    #x = np.rollaxis(x, 0, 2) 
    x = np.expand_dims(x, axis=-1)
    x = x.astype(np.float32)
    return x

In [ ]:
import os, os.path
imgs = []
path2 = "D:/Kanwil Riau/Updating NPGT Perkebunan/OLAH TIM/RIAU/Run/Predict"
valid_images = [".tif"]
for f in os.listdir(path2):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    imgs.append(os.path.join(path2,f))
imgs

In [ ]:
from tqdm import tqdm
import numpy as np
import rasterio as rio

for k in tqdm(imgs):
    test = read_image2(k)  # Fungsi untuk membaca citra

    t0 = rio.open(k)
    meta = t0.meta

    # Memperbarui metadata untuk menyimpan 1 saluran untuk 7 kelas
    meta.update({
        'driver': 'GTiff', 
        'width': 1024, 
        'height': 1024, 
        'count': 1,  # Hanya 1 saluran untuk menyimpan label kelas
        'dtype': 'int16',  # Ubah dtype menjadi int16
        'nodata': -32768
    }) 

    name = k.split("\\")[-1].split(".")[0]
    
    # Melakukan prediksi, pastikan model menghasilkan output dengan 7 kelas
    y_pred = model.predict(np.expand_dims(test, axis=0))[0]  # Ini seharusnya (128, 128, 7)
    
    # Mengambil hasil prediksi kelas dengan argmax untuk mendapatkan label kelas
    y_pred_classes = np.argmax(y_pred, axis=-1)  # Mengambil kelas dengan probabilitas tertinggi

    # Pastikan y_pred_classes memiliki dimensi (128, 128)
    
    # Inisialisasi array untuk menyimpan hasil kelas
    all_images = np.zeros((512, 512), dtype=np.int16)  # Hanya satu saluran untuk kelas

    # Mengisi all_images dengan kelas
    all_images[:, :] = y_pred_classes  # Simpan kelas langsung

    name = k.split("/")[-1].split(".")
    
    """ Extracting the name and extension of the image and the mask. """ 
    image_name = name[0] 
    image_extn = name[1] 
    
    tmp_img_name = f"{image_name}_results.{image_extn}" 
    image_path = os.path.join(path2, "results", tmp_img_name) 
    
    with rio.open(image_path, 'w', **meta) as dst: 
        dst.write(all_images[np.newaxis, :, :])  # Menulis dengan menambahkan dimensi saluran
